<a href="https://colab.research.google.com/github/PriyankaB26/Youtube-sentiment-Analyzer/blob/main/modelsentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd

In [21]:
from google.colab import files
file.open("/content/drive/My\ Drive/IMDB Dataset.csv")

NameError: name 'file' is not defined

In [ ]:
# Load the uploaded file
df = pd.read_csv('IMDB Dataset.csv')

# New Section

In [22]:
import re
import string

def clean_text(text):
    # Lowercasing the text
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply the cleaning function to the reviews
df['cleaned_review'] = df['review'].apply(clean_text)

# Preview cleaned data
print(df['cleaned_review'].head())



0    one of the other reviewers has mentioned that ...
1    a wonderful little production br br the filmin...
2    i thought this was a wonderful way to spend ti...
3    basically theres a family where a little boy j...
4    petter matteis love in the time of money is a ...
Name: cleaned_review, dtype: object


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   review          50000 non-null  object
 1   sentiment       50000 non-null  object
 2   cleaned_review  50000 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)

# Fit and transform the cleaned reviews
X = tfidf.fit_transform(df['cleaned_review']).toarray()

# Encode the sentiment labels (assuming 'positive' and 'negative')
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['sentiment'])

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Initialize the model
model = MultinomialNB()

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 0.8516
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85      6157
    positive       0.86      0.85      0.85      6343

    accuracy                           0.85     12500
   macro avg       0.85      0.85      0.85     12500
weighted avg       0.85      0.85      0.85     12500



In [29]:
# Assuming the model, tfidf, and le are already available from the training process

# Function to fetch and analyze YouTube comments
def analyze_youtube_video_sentiment(video_id, model, tfidf, le):
    # Fetch comments
    comments = video_comments(video_id)

    # Clean the comments
    cleaned_comments = [clean_text(comment) for comment in comments]

    # Transform comments into the TF-IDF feature space
    X_comments = tfidf.transform(cleaned_comments).toarray()

    # Predict sentiments
    predicted_sentiments = model.predict(X_comments)

    # Convert predictions to sentiment labels
    sentiment_labels = le.inverse_transform(predicted_sentiments)

    # Calculate sentiment percentages
    positive_percentage, negative_percentage, neutral_percentage = calculate_sentiment_percentages(sentiment_labels)

    # Display results
    print(f"Positive: {positive_percentage:.2f}%")
    print(f"Negative: {negative_percentage:.2f}%")
    print(f"Neutral: {neutral_percentage:.2f}%")

    return sentiment_labels




Positive: 50.45%
Negative: 49.55%
Neutral: 0.00%


In [33]:
# Example usage: replace with the actual video ID
video_link = input("Enter the video link: ")  # Replace with the actual video ID
video_id = video_link.split("=")[1]
sentiment_results = analyze_youtube_video_sentiment(video_id, model, tfidf, le)


Enter the video link: https://www.youtube.com/watch?v=3xIVCRoNCXg
Positive: 41.01%
Negative: 58.99%
Neutral: 0.00%


In [40]:
# Example usage: replace with the actual video ID
video_link = input("Enter the video link: ")  # Replace with the actual video ID
video_id = video_link.split("=")[1]
sentiment_results = analyze_youtube_video_sentiment(video_id, model, tfidf, le)


Enter the video link: https://www.youtube.com/watch?v=EVDsImdq4Yg
Positive: 41.08%
Negative: 58.92%
Neutral: 0.00%


In [41]:
# Example usage: replace with the actual video ID
video_link = input("Enter the video link: ")  # Replace with the actual video ID
video_id = video_link.split("=")[1]
sentiment_results = analyze_youtube_video_sentiment(video_id, model, tfidf, le)


Enter the video link: https://www.youtube.com/watch?v=pX2zvfD6GCY
Positive: 36.00%
Negative: 64.00%
Neutral: 0.00%
